<a href="https://colab.research.google.com/github/Yiting916/ML/blob/main/GCP_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 設置客戶端 SDK

In [ ]:
import os
from google.cloud import bigquery

# 設置 Google Cloud 認證
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r"金鑰.json" # 上傳金鑰json檔


# 初始化 BigQuery 客戶端
client = bigquery.Client()
print('connection done')


connection done


# 建立資料集

In [ ]:
# 定義資料集參數
dataset_id = 'tibame_gad245_14_00312_dataset_us'  # 替換為想要建立的資料集名稱
project_id = 'tibame-gad245-14-0305'   # 替換為 Google Cloud 專案 ID
dataset_ref = client.dataset(dataset_id)

# 設定資料集的描述和其他選項
dataset = bigquery.Dataset(dataset_ref)
dataset.description = 'This is a new dataset created from Python.'
dataset.location = 'US'  # 設定資料集的地區位置

# 建立資料集
dataset = client.create_dataset(dataset, timeout=30)  # 可設定超時時間
print(f"dataset: {dataset.dataset_id} ,created")


Conflict: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/tibame-gad245-14-0305/datasets?prettyPrint=false: Already Exists: Dataset tibame-gad245-14-0305:tibame_gad245_14_00312_dataset_us

# 建立資料表

In [ ]:
# 定義資料集和資料表
# dataset_id = 'tibame_gad245_14_00312_dataset_us'
table_id = 'age_table'
schema = [
    bigquery.SchemaField("name", "STRING"),
    bigquery.SchemaField("age", "INTEGER"),
]

# 定義表格參數
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=schema)

# 建立資料表
table = client.create_table(table)
print(f"table: {table.table_id} ,created")

Conflict: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/tibame-gad245-14-0305/datasets/tibame_gad245_14_00312_dataset_us/tables?prettyPrint=false: Already Exists: Table tibame-gad245-14-0305:tibame_gad245_14_00312_dataset_us.age_table

# 寫入資料

In [ ]:
# 定義資料集和表資料表名稱
dataset_id = 'tibame_gad245_14_00312_dataset_us'
table_id = 'age_table'

# 定義資料
rows_to_insert = [
    {"name": "Alice", "age": 25},
    {"name": "Bob", "age": 35},
    {"name": "Summer", "age": 40},
    {"name": "Apple", "age": 18}
]

# 插入資料
errors = client.insert_rows_json(f"{dataset_id}.{table_id}", rows_to_insert)

if errors == []:
    print("data insert successfully")
else:
    print(f"error: {errors}")



data insert successfully


# 查詢資料

In [ ]:
# 定義查詢語句
query = """
    SELECT name, age
    FROM `tibame_gad245_14_00312_dataset_us.age_table`
    WHERE age > 30
"""

# 執行查詢
query_job = client.query(query)

# 獲取結果
results = query_job.result()

# 印出結果
for row in results:
    print(f"name: {row.name}, age: {row.age}")

name: Bob, age: 35
name: Summer, age: 40


# 刪除資料表

In [ ]:
# 定義資料集和資料表名稱
dataset_id = 'tibame_gad245_14_00312_dataset_us'
table_id = 'age_table'

# 定義資料表參數
table_ref = client.dataset(dataset_id).table(table_id)

# 刪除資料表
client.delete_table(table_ref)
print(f"table: {table_id} ,deleted")


table: age_table ,deleted


# 建立tenserflow模型

In [ ]:
# 定義 SQL 查詢以創建模型
query = """
CREATE OR REPLACE MODEL `tibame-gad245-14-0305.tibame_gad245_14_00312_dataset_us.imported_tf_model`
OPTIONS (MODEL_TYPE='TENSORFLOW',
         MODEL_PATH='gs://cloud-training-demos/txtclass/export/exporter/1549825580/*')
"""

# 執行 SQL 查詢以創建模型
query_job = client.query(query)
query_job.result()  # 等待查詢完成

print("TensorFlow model import BigQuery ML")


TensorFlow model import BigQuery ML


# 測試tensorflow模型資料

In [ ]:
# 定義 SQL 查詢以使用模型進行預測，只取五筆非 NULL 的資料
query = """
SELECT *
FROM ML.PREDICT(MODEL `tibame-gad245-14-0305.tibame_gad245_14_00312_dataset_us.imported_tf_model`,
  (SELECT title AS input
   FROM `bigquery-public-data.hacker_news.full`
   WHERE title IS NOT NULL
   LIMIT 5)  -- 限制結果為五筆資料
)
"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

# 印出預測結果
for row in results:
    print(row)



Row(([1.0, 3.44244477675026e-25, 2.3777851297082306e-14], 'Emacs 25.1 on Microsoft Windows will support daemon mode'), {'dense_1': 0, 'input': 1})
Row(([7.772989474297276e-10, 0.9999434947967529, 5.6485274399165064e-05], 'VR will be won and mainstreamed at the low-end'), {'dense_1': 0, 'input': 1})
Row(([0.005399384535849094, 2.337307647773401e-12, 0.9946006536483765], '25 ways to promote your android app for free'), {'dense_1': 0, 'input': 1})
Row(([0.9687144756317139, 0.003372062696143985, 0.027913538739085197], 'Improving browser security by OpenBSD'), {'dense_1': 0, 'input': 1})
Row(([7.101416233079279e-15, 0.0008984644082374871, 0.9991015195846558], 'Tinder Plus: First step into the world of freemium'), {'dense_1': 0, 'input': 1})


# 刪除資料集

In [ ]:
# 定義資料集名稱
dataset_id = 'tibame_gad245_14_00312_dataset_us'

# 定義資料集參數
dataset_ref = client.dataset(dataset_id)

# 刪除資料集
client.delete_dataset(dataset_ref, delete_contents=True)
print(f"dataset: {dataset_id} ,deleted")

dataset: tibame_gad245_14_00312_dataset_us ,deleted
